## Assignment Problem

In [1]:
import sys
import docplex.mp

### Step 2: Model the data

The input data is the number of objects (and boxes) _N_, and their positions in the (x,y) plane.

### Step 3: Prepare the data

We use Euclidean distance to compute the distance between an object and its assigned box.



In [3]:
from math import sqrt

N = 3
var_range = range(1, N)
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Windows 64bit
* Python version 3.7.9, located at: E:\DevelopmentEnvironments\Anaconda3\envs\optimization\python.exe
* docplex is present, version is 2.19.202
* CPLEX library is present, version is 12.10.0.0, located at: C:\Program Files\IBM\ILOG\CPLEX_Studio_Community1210\cplex\python\3.7\x64_win64
* pandas is not available


#### Create the DOcplex model
The model contains all the business constraints and defines the objective.

In [17]:
from docplex.mp.model import Model
mdl = Model("Linear Optimization")

In [18]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [50, 100, 90, 100],
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Variables
# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = mdl.binary_var(name="x_%d_%d" %(i, j))

# Constraints
# Each worker is assigned to at most 1 task.
mdl.add_constraints(mdl.sum(x[i, j] for j in range(num_tasks)) <= 1 for i in range(num_workers))

# Each task is assigned to exactly one worker.
mdl.add_constraints(mdl.sum(x[i, j] for i in range(num_workers)) == 1 for j in range(num_tasks))        

[docplex.mp.LinearConstraint[](x_0_0+x_1_0+x_2_0+x_3_0+x_4_0,EQ,1),
 docplex.mp.LinearConstraint[](x_0_1+x_1_1+x_2_1+x_3_1+x_4_1,EQ,1),
 docplex.mp.LinearConstraint[](x_0_2+x_1_2+x_2_2+x_3_2+x_4_2,EQ,1),
 docplex.mp.LinearConstraint[](x_0_3+x_1_3+x_2_3+x_3_3+x_4_3,EQ,1)]

In [19]:
mdl.print_information()

Model: Linear Optimization
 - number of variables: 20
   - binary=20, integer=0, continuous=0
 - number of constraints: 9
   - linear=9
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [20]:
print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Linear Optimization

Minimize
 obj:
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 + x_0_3 <= 1
 c2: x_1_0 + x_1_1 + x_1_2 + x_1_3 <= 1
 c3: x_2_0 + x_2_1 + x_2_2 + x_2_3 <= 1
 c4: x_3_0 + x_3_1 + x_3_2 + x_3_3 <= 1
 c5: x_4_0 + x_4_1 + x_4_2 + x_4_3 <= 1
 c6: x_0_0 + x_1_0 + x_2_0 + x_3_0 + x_4_0 = 1
 c7: x_0_1 + x_1_1 + x_2_1 + x_3_1 + x_4_1 = 1
 c8: x_0_2 + x_1_2 + x_2_2 + x_3_2 + x_4_2 = 1
 c9: x_0_3 + x_1_3 + x_2_3 + x_3_3 + x_4_3 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_0_3 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_1_3 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1
 0 <= x_2_3 <= 1
 0 <= x_3_0 <= 1
 0 <= x_3_1 <= 1
 0 <= x_3_2 <= 1
 0 <= x_3_3 <= 1
 0 <= x_4_0 <= 1
 0 <= x_4_1 <= 1
 0 <= x_4_2 <= 1
 0 <= x_4_3 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_0_3 x_1_0 x_1_1 x_1_2 x_1_3 x_2_0 x_2_1 x_2_2 x_2_3 x_3_0
 x_3_1 x_3_2 x_3_3 x_4_0 x_4_1 x_4_2 x_4_3
End



#### Express the objective

* The objective is to minimize the total distance between each object and its storage box.

In [23]:
# minimize total displacement
mdl.minimize(mdl.sum(costs[i][j] * x[i,j] for i in range(num_workers) for j in range(num_tasks)) )

In [24]:
mdl.export_as_lp(path='abc.lp')

'abc.lp'

#### Solve the model


In [25]:
mdl.print_information()

assert mdl.solve(), "!!! Solve of the model fails"

Model: Linear Optimization
 - number of variables: 20
   - binary=20, integer=0, continuous=0
 - number of constraints: 9
   - linear=9
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [26]:
mdl.report()
d1 = mdl.objective_value
mdl.print_solution()

* model Linear Optimization solved with objective = 265.000
objective: 265
  x_0_3=1
  x_1_2=1
  x_2_1=1
  x_3_0=1
